In [1]:
from optimizers.gradient import GradientOptimizer, RMSD_loss_fn
from utils.general import CustomIgFold, FVbinder

/home/ubuntu/miniconda3/lib/python3.12/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
CIF = CustomIgFold()

Loading /home/ubuntu/miniconda3/lib/python3.12/site-packages/igfold/trained_models/IgFold/igfold_3.ckpt...
Loading /home/ubuntu/miniconda3/lib/python3.12/site-packages/igfold/trained_models/IgFold/igfold_5.ckpt...
Loading /home/ubuntu/miniconda3/lib/python3.12/site-packages/igfold/trained_models/IgFold/igfold_2.ckpt...
Loading /home/ubuntu/miniconda3/lib/python3.12/site-packages/igfold/trained_models/IgFold/igfold_1.ckpt...


In [3]:
start_vh = 'QVQLVQSGAEVKKPGSSVKVSCKASGGTFSSYRMHWVRQAPGQGLEWMGYINPSTGYTEYNQKFKDRVTITADESTNTAYMELSSLRSEDTAFYFCAGGGGVFDYEYNGGLVTVSS'
start_vl = 'DIQMTQSPSTLSASVGDRVTITCSASSSISYMHWYQQKPGKAPKLLMYTTSNLASGVPSRFIGSGSGTEFTLTISSLQPDDFATYYCHQRSTYPLTFGQGTKVEVK'
start = FVbinder(start_vh, start_vl)

In [4]:
heavy_sequence = "QVQLQQSGAELAKPGASVKMSCKASGYTFTSYRMHWVKQRPGQGLEWIGYINPSTGYTEYNQKFKDKATLTADKSSSTAYMQLSSLTFEDSAVYYCARGGGVFDYWGQGTTLTVSS" #@param {type:"string"}
light_sequence = "QIVLTQSPAIMSASPGEKVTITCSASSSISYMHWFQQKPGTSPKLWIYTTSNLASGVPARFSGSGSGTSYSLTISRMEAEDAATYYCHQRSTYPLTFGSGTKLELK" #@param {type:"string"}
target = FVbinder(vh=heavy_sequence, vl=light_sequence)
CIF.fold(target)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
/home/ubuntu/miniconda3/lib/python3.12/site-packages/igfold/utils/coordinates.py:18: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at ../aten/src/ATen/native/Cross.cpp:62.)
  n_vec = (b_coord - a_coord).expand(bc_vec.shape).cross(bc_vec)


In [5]:
humanized_vh = "QVQLVQSGAEVKKPGSSVKVSCKASGYTFTSYRMHWVRQAPGQGLEWIGYINPSTGYTEYNQKFKDKATITADESTNTAYMELSSLRSEDTAVYYCARGGGVFDYWGQGTLVTVSS"
humanized_vl = "DIQMTQSPSTLSASVGDRVTITCSASSSISYMHWYQQKPGKAPKLLIYTTSNLASGVPARFSGSGSGTEFTLTISSLQPDDFATYYCHQRSTYPLTFGQGTKVEVK"
humanized = FVbinder(vh=humanized_vh, vl=humanized_vl)

In [6]:
optimizer = GradientOptimizer(lr=0.001, sub_infinity=1, start=start, target=target, validation=False, verbose=True, tensorboard=True)

In [7]:
optimizer.step(1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

RMSD Loss: tensor([0.0074], device='cuda:0', grad_fn=<DivBackward0>)
Species Loss: 1.8505158424377441
Log-likelyhood Loss: 2.4331912994384766
Full Loss: tensor([0.4445], device='cuda:0', grad_fn=<AddBackward0>)

---
RMSD Loss: tensor([0.0074], device='cuda:0', grad_fn=<DivBackward0>)
Species Loss: 1.8505051136016846
Log-likelyhood Loss: 2.433220863342285
Full Loss: tensor([0.4448], device='cuda:0', grad_fn=<AddBackward0>)

---
RMSD Loss: tensor([0.0074], device='cuda:0', grad_fn=<DivBackward0>)
Species Loss: 1.8504948616027832
Log-likelyhood Loss: 2.4332330226898193
Full Loss: tensor([0.4449], device='cuda:0', grad_fn=<AddBackward0>)

---
RMSD Loss: tensor([0.0074], device='cuda:0', grad_fn=<DivBackward0>)
Species Loss: 1.8504835367202759
Log-likelyhood Loss: 2.433251142501831
Full Loss: tensor([0.4451], device='cuda:0', grad_fn=<AddBackward0>)

---
RMSD Loss: tensor([0.0074], device='cuda:0', grad_fn=<DivBackward0>)
Species Loss: 1.8504712581634521
Log-likelyhood Loss: 2.4332768917083

In [22]:
optimizer.generator.antiberty_tokenizer.decode(optimizer.generator.logits[0].argmax(-1), skip_special_tokens=True)

'P T N L D Q W W Y M W D Y W L G Q I Y Q I C L A Y G G T F S S Y G M H N D C E Q M V F V L N Y N Y S I N P S T G Y M E Q N N N M K K N V V V V A E E A T N T T Y M E L V W L R P E S A A F Y F V R G G G G V F D Y E Y N K G V V H H C T'

In [8]:
print(optimizer.best_valid_binder)

FVbinder(
vh='NYSGSSVKVWCVASGGTFSSYRMHTRASASNMGLNNTGYQNPSTGYTEYNQYWKDRVYIAADESTNTAYMELSSLRSEDTAFYFCAGGGGVFDYEYNGGLVVVSS',
vl='DIQMTQSSGTLKASVGDRRTITCSASSSISYMHWYNGKPGRATNLISYTTSNLASGVPSRFIGSGSGTEFTLTISPLCPDRFCTCYCHQRSTYPLTFGQGTKVEVI'
)


In [14]:
len(optimizer.best_valid_binder.vh.seq)

105

In [15]:
len(humanized_vh)

116

In [9]:
start.vh.align(optimizer.best_valid_binder.vh, humanized.vh)

QVQLVQSGAEVKKPGSSVKVSCKASGGTFSSYRMHWVRQAPGQGLEWMGYINPSTGYTEYNQKFKDRVTITADESTNTAYMELSSLRSEDTAFYFCAGGGGVFDYEYNGGLVTVSS
..............||||||.|.||||||||||||....|...||...||.|||||||||||.+||||.|.|||||||||||||||||||||||||||||||||||||||||.|||
-----------NYSGSSVKVWCVASGGTFSSYRMHTRASASNMGLNNTGYQNPSTGYTEYNQYWKDRVYIAADESTNTAYMELSSLRSEDTAFYFCAGGGGVFDYEYNGGLVVVSS
..............||||||.|.|||.||+|||||....|...||...||.|||||||||||.+||+..|.|||||||||||||||||||||.|+||.|||||||...|.||.|||
QVQLVQSGAEVKKPGSSVKVSCKASGYTFTSYRMHWVRQAPGQGLEWIGYINPSTGYTEYNQKFKDKATITADESTNTAYMELSSLRSEDTAVYYCARGGGVFDYWGQGTLVTVSS
                         ^^^^^^^                   ^^^^^^                                         ^^^^^^^           

In [10]:
start.vl.align(optimizer.best_valid_binder.vl, humanized.vl)

DIQMTQSPSTLSASVGDRVTITCSASSSISYMHWYQQKPGKAPKLLMYTTSNLASGVPSRFIGSGSGTEFTLTISSLQPDDFATYYCHQRSTYPLTFGQGTKVEVK
|||||||..||.||||||.||||||||||||||||..|||+|..|+.||||||||||||||||||||||||||||.|.||.|.|.||||||||||||||||||||.
DIQMTQSSGTLKASVGDRRTITCSASSSISYMHWYNGKPGRATNLISYTTSNLASGVPSRFIGSGSGTEFTLTISPLCPDRFCTCYCHQRSTYPLTFGQGTKVEVI
|||||||..||.||||||.||||||||||||||||..|||+|..|+.|||||||||||+||.|||||||||||||.|.||.|.|.||||||||||||||||||||.
DIQMTQSPSTLSASVGDRVTITCSASSSISYMHWYQQKPGKAPKLLIYTTSNLASGVPARFSGSGSGTEFTLTISSLQPDDFATYYCHQRSTYPLTFGQGTKVEVK
                       ^^^^^^^^^^               ^^^^^^^                                ^^^^^^^^^          

In [ ]:
humanized.vh.align(optimizer.best_valid_binder.vh)

In [18]:
humanized.vh.align(optimizer.best_valid_binder.vh)

EVQLQESGPGLVKPSETLSLTCAVSGYSITSGYSWNWIRQPPGNKLEWMGYVNNDGLNNYNPSLKSRITITRDTSKNQFSLKFSSVTAADTAVYFCTLYGYDGAWFTYWGQGTLVTVSS
+||||||||.|||||.+|||||+|+||||||||.||||||.||||||||||.|||||||||||||+||+||||||||||.|+|.|||..|||.||||.||||||||||||||||||||+
DVQLQESGPSLVKPSHSLSLTCSVTGYSITSGYHWNWIRQFPGNKLEWMGYHNNDGLNNYNPSLKNRISITRDTSKNQFFLRFHSVTTEDTATYFCTHYGYDGAWFTYWGQGTLVTVSA
                         ^^^^^^^^                   ^^^^^                                         ^^^^^^^^^^           

In [19]:
humanized.vl.align(optimizer.best_valid_binder.vl)

DIVLTQSPSSLSASVGDRVTITCLASQTIGTWLAWYQQKPGKSPQLLIYAATSLADGVPSRFSGSGSGTKFTLTISSLQPEDFATYYCQQLYSTPYTFGGGTKVEIK
||.+||||+|.|||+|+|||||||||||||||||||.|||||+|.||||||||||||||||||||||||.|+..|||||.|||.+||||||||||||||.|||+|||
DIHMTQSPASQSASLGERVTITCLASQTIGTWLAWYHQKPGKAPNLLIYAATSLADGVPSRFSGSGSGTNFSFNISSLQAEDFVSYYCQQLYSTPYTFGNGTKLEIK
                       ^^^^^^^^^^^               ^^^^^^^                                ^^^^^^^^^          

In [8]:
from abnumber import Chain

In [9]:
seq = "QRQLEQTGSCVKKSGSMVLVSSVASGGTFSSYRMHYVTNAPGNVLSIMVYINPSTGYTETNPKMKDRVTIQADEPTNTFYMEVSPIIPLPQSFYFCAKGGGVFDYERRGGLVNVSS"

In [10]:
chain = Chain(seq, scheme="chothia", assign_germline=True)

In [11]:
chain.species

'human'

In [ ]:
optimizer.best_valid_binder.vh.renumber("imgt").find_human_germlines

In [8]:
def v_gene_identity(sequence):
    c = Chain(sequence, scheme="imgt")
    v_germline_chains, _ = c.find_human_germlines(limit=1)
    top_v = v_germline_chains[0]

    num_v_matches = sum(top_v.positions.get(pos) == aa for pos, aa in c)

    v_identity = num_v_matches / len(top_v)
    return v_identity

In [10]:
v_gene_identity(seq)

0.4897959183673469

In [ ]:
v_gene_identity(optimizer.best_valid_binder.vl.seq)

0.49473684210526314

In [ ]:
chain = Chain(seq, 'imgt')

v_germline_chains, j_germline_chains = chain.find_human_germlines(limit=1)
top_v, top_j = v_germline_chains[0], j_germline_chains[0]

num_v_matches = sum(top_v.positions.get(pos) == aa for pos, aa in chain)
num_j_matches = sum(top_j.positions.get(pos) == aa for pos, aa in chain)

v_identity = num_v_matches / len(top_v)
j_identity = num_j_matches / len(top_j)
full_identity = (num_v_matches + num_j_matches) / len(chain)

In [ ]:
v_identity

0.5773195876288659

In [2]:
import torch
import torch.nn.functional as F

In [3]:
logits = torch.rand(2, 227, 25).transpose(1, 2)
pred_logits = torch.rand(2, 227, 25).transpose(1, 2)
F.cross_entropy(pred_logits, logits.detach(), reduction="mean")

tensor(40.5223)